In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [11]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
from datasets import DatasetDict

In [14]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
         # Efficiently select the first 10 rows using `select()`
        
        ### because I am using cpu ###

        # Select only the first three rows from the "train" split
        train_dataset = dataset_samsum["train"].select([0, 1])
        
        # Select only the first three rows from the "validation" split
        validation_dataset = dataset_samsum["validation"].select([0])
        
        # Create a DatasetDict with "train" and "validation" keys
        dataset_samsum = DatasetDict({
            "train": train_dataset,
            "validation": validation_dataset,
        })
        #dataset_samsum = dataset_samsum.select(range(3)) 
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-03-03 20:43:25,906: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-03 20:43:25,926: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 20:43:25,931: INFO: common: created directory at: artifacts]
[2024-03-03 20:43:25,932: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 1/1 [00:00<00:00, 108.34 examples/s]
